# Retrieval Augmented Generation with Amazon Bedrock - Retrieving Data Automatically from APIs

> *PLEASE NOTE: This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

---

Throughout this workshop so far, we have been working with unstructured text retrieval via semantic similarity search. However, another important type of retrieval which customers can take advantage of with Amazon Bedrock is **structured data retrieval** from APIs. Structured data retrieval is extremely useful for augmenting LLM applications with up to date information which can be retrieved in a repeatable manner, but the outputs are always changing. An example of a question you might ask an LLM which uses this type of retrieval might be "How long will it take for my Amazon.com order containing socks to arrive?". In this notebook, we will show how to integrate an LLM with a backend API service which has the ability to answer a user's question through RAG.

Specifically, we will be building a tool which is able to tell you the weather based on natural language. This is a fairly trivial example, but it does a good job of showing how multiple API tools can be used by an LLM to retrieve dynamic data to augment a prompt. Here is a visual of the architecture we will be building today.

![api](./images/api.png)

Let's get started!

---
## Setup

In [1]:
import sys
import os
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils.environment_validation import validate_environment, validate_model_access
validate_environment()

Validating base environment
Base environment validated successfully


Validating lab environment from requirements.txt ✨

ENVIRONMENT STATUS
✅  langchain==0.1.14 is installed
✅  langchain-aws is installed
✅  sqlalchemy is installed
✅  pypdf>=3.8,<4 is installed
✅  datasets is installed
✅  matplotlib is installed
✅  pymupdf  is installed
✅  xmltodict==0.13.0 is installed
✅  duckduckgo-search is installed
✅  yfinance is installed
✅  pandas-datareader is installed
✅  pysqlite3 is installed

All required libraries are installed.🎉
You may proceed with the lab! 🚀

In [2]:
required_models = [
    "amazon.titan-embed-text-v1",
    "anthropic.claude-3-sonnet-20240229-v1:0",
    "anthropic.claude-3-haiku-20240307-v1:0",
]
validate_model_access(required_models)

MODEL ACCESS STATUS
✅  amazon.titan-embed-text-v1 is accessible
✅  anthropic.claude-3-sonnet-20240229-v1:0 is accessible
✅  anthropic.claude-3-haiku-20240307-v1:0 is accessible

All required models are accessible.🎉
You may proceed with the lab! 🚀

In [3]:
import json

import boto3
import botocore

import xmltodict
from rich import print as rprint

from utils import bedrock
from utils.prompt_utils import extract_docstring_info, construct_format_tool_for_claude_prompt, prompts_to_messages_converse

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


---
## Defining the API Tools

The first thing we need to do for our LLM is define the tools it has access to. In this case we will be defining local Python functions, but it is important to note that these could be any type of application service. Examples of what these tools might be on AWS include...

* An AWS Lambda function
* An Amazon RDS database connection
* An Amazon DynamnoDB table
  
More generic examples include...

* REST APIs
* Data warehouses, data lakes, and databases
* Computation engines

In this case, we define two tools which reach external APIs below with two python functions
1. the ability to retrieve the latitude and longitude of a place given a natural language input
2. the ability to retrieve the weather given an input latitude and longitude

In [4]:
import requests

def get_weather(latitude: str, longitude: str):
    """
    Fetches and returns the current weather for a given latitude and longitude from the Open-Meteo API.

    Args:
        latitude (str): The latitude of the location to fetch the weather for.
        longitude (str): The longitude of the location to fetch the weather for.

    Returns:
        dict: A dictionary containing the current weather data for the given location.
    """
    
    
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    return response.json()

def get_lat_long(place: str):
    
    """
    Fetches and returns the latitude and longitude of a given place from the Nominatim OpenStreetMap API.

    Args:
        place (str): The name of the place to fetch the latitude and longitude for.

    Returns:
        dict: A dictionary containing the latitude and longitude of the given place, or None if the place was not found.
    """
    
    url = "https://nominatim.openstreetmap.org/search"
    params = {'q': place, 'format': 'json', 'limit': 1}
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, params=params, headers=headers).json()
    if response:
        lat = response[0]["lat"]
        lon = response[0]["lon"]
        return {"latitude": lat, "longitude": lon}
    else:
        return None

The [Converse](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html) api within Amazon Bedrock supports native tool usage for an array of Models from Anthropic, Mistral, and Cohere. We will use this api to have the model dynamically fetch the weather based on the user's input.

To help make the solution a bit more robust, we will create pydantic models for each function input. This will help us validate the input data that will be generated by the model and will also produce an json input schema that can be used to construct the tool specification

In [5]:
from pydantic import BaseModel, Field

In [6]:
class GetLocationInput(BaseModel):
    place: str = Field(..., description="The name of the place to fetch the latitude and longitude for.")

class GetWeatherInput(BaseModel):
    latitude: str = Field(..., description="The latitude of the location to fetch the weather for.")
    longitude: str = Field(..., description="The longitude of the location to fetch the weather for.")

The code below will create the tool specification for the two tools we will be using in this example. For more details see [here](https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use.html). 

In [12]:
tools = []
tool_lookup = {}
for func, inp_obj in zip(
    [get_lat_long, get_weather], [GetLocationInput, GetWeatherInput]
):
    tool_lookup[func.__name__] = {"func": func, "input": inp_obj}
    tools.append(
        {
            "toolSpec": {
                "name": func.__name__,
                "description": func.__doc__,
                "inputSchema": {"json": inp_obj.model_json_schema()}, # much easier than having to write the schema manually especially for more complex functions
            }
        }
    )
tool_config = {"tools": tools}


In [23]:
# import pprint
# pp =pprint.PrettyPrinter()
# pp.pprint(tool_config)
print(json.dumps(tool_config, indent=2))

{
  "tools": [
    {
      "toolSpec": {
        "name": "get_lat_long",
        "description": "\n    Fetches and returns the latitude and longitude of a given place from the Nominatim OpenStreetMap API.\n\n    Args:\n        place (str): The name of the place to fetch the latitude and longitude for.\n\n    Returns:\n        dict: A dictionary containing the latitude and longitude of the given place, or None if the place was not found.\n    ",
        "inputSchema": {
          "json": {
            "properties": {
              "place": {
                "description": "The name of the place to fetch the latitude and longitude for.",
                "title": "Place",
                "type": "string"
              }
            },
            "required": [
              "place"
            ],
            "title": "GetLocationInput",
            "type": "object"
          }
        }
      }
    },
    {
      "toolSpec": {
        "name": "get_weather",
        "description": "\n    F

---
## Executing the RAG Workflow

Armed with our prompt and structured tools, we can now write an orchestration function which will iteratively step through the logical tasks to answer a user question. In the cell below we use the `invoke_model` function to generate a response with Claude and the `single_retriever_step` function to iteratively call tools when the LLM tells us we need to. The general flow works like this...

1. The user enters an input to the application
2. The user input is merged with the original prompt and sent to the LLM to determine the next step
3. If the LLM knows the answer, it will answer and we are done. If not, go to next step 4.
4. The LLM will determine which tool to use to answer the question.
5. We will use the tool as directed by the LLM and retrieve the results.
6. We provide the results back into the original prompt as more context.
7. We ask the LLM the next step or if knows the answer.
8. Return to step 3.

We can orchestrate the entire workflow through a simple while loop where we continuously invoke the LLM until it has enough information to provide the final answer.

In [8]:
llm_modelId = "anthropic.claude-3-haiku-20240307-v1:0" # model ID for the LLM model to use

# user's question
prompt = "What is the weather in New York?"
messages = prompts_to_messages_converse([{"role": "user", "text_prompt": prompt}])

# system message to define the role and behavior of the assistant
system_message = """
You are a meterological assistant.
Your job is to answer weather-related questions. You have access to tools to look up weather information for a given location.
You may need to utilize multiple tools to answer the user's question.
Place any intermediate thoughts into <thinking> tags
When you have the final answer, provide it in <answer> tags
"""


inference_config = {"temperature": 0.0}

rprint(f"[bold #219ebc]Initial prompt:[/bold #219ebc] [#ffb703]{prompt}[/#ffb703]")


# The execution workflow will run in a loop until the assistant provides a final answer
while True:

    # invoke the model with the messages (conversation state), system prompt, and tool configuration
    response = boto3_bedrock.converse(
        modelId=llm_modelId,
        messages=messages,
        system=[{"text": system_message}],
        inferenceConfig=inference_config,
        toolConfig=tool_config,
        # additionalModelRequestFields=additional_model_fields
    )

    response_messages = response["output"]["message"]
    response_content = response_messages["content"]
    messages.append(response_messages)

    # if the model determines that a tool should be used, it will provide "tool_use" as the stop reason
    if response["stopReason"] == "tool_use":

        # model may respond with both the tool_use response and some intermediate thoughts
        text_content = next((m['text'] for m in response_content if 'text' in m), "")
        
        tool_use_content = [m for m in response_content if "toolUse" in m][0]

        # from the tool_use response, extract the tool name, and the input to the tool
        tool_use_id = tool_use_content["toolUse"]["toolUseId"]
        tool_name = tool_use_content["toolUse"]["name"]
        tool_input = tool_use_content["toolUse"]["input"]
        
        rprint(f"[bold #219ebc]Intermediate response:[/bold #219ebc] [#ffb703]{text_content}[#ffb703]")
        rprint(f"[bold #219ebc]Tool name:[/bold #219ebc] [#ffb703]{tool_name}[#ffb703]")
        rprint(f"[bold #219ebc]Tool input:[/bold #219ebc] [#ffb703]{tool_input}[#ffb703]")

        try:
            # get the function to invoke
            function = tool_lookup[tool_name]["func"]
            
            # get the pydanitc model to parse the input
            function_object = tool_lookup[tool_name]["input"]
            
            # parse the input
            tool_input_obj = function_object.parse_obj(tool_input)
            
            # invoke the function with the parsed input
            result = function(**tool_input_obj.dict())
            rprint(f"[bold #219ebc]Tool output:[/bold #219ebc] [#ffb703]{result}[#ffb703]")
            
            
            # we have to attach the tool response to the messages (conversation history) to continue the conversation
            tool_response_message = prompts_to_messages_converse(
                [
                    {
                        "role": "user",
                        "tool_use_id": tool_use_id,
                        "text_prompt": f"The output of {tool_name} is {result}",
                        "tool_status": "success",
                    }
                ]
            )
            messages.extend(tool_response_message)

        # in case there is an invocation error, we will attach the error message to the messages so that the model can make corrections
        except Exception as e:
            error = str(e)
            tool_response_message = prompts_to_messages_converse(
                [
                    {
                        "role": "user",
                        "tool_use_id": tool_use_id,
                        "text_prompt": result,
                        "tool_status": "error",
                    }
                ]
            )
            messages.extend(tool_response_message)
            
    # if generation stopped for any other reason
    else:
        text_content = [m for m in response_content if "text" in m][0]
        result = text_content["text"]
        if "<answer>" in result:
            break

final_answer = result.replace("<answer>", "").replace("</answer>", "").strip()
rprint(f"[bold #219ebc]Final answer:[/bold #219ebc] [#ffb703]{final_answer}[#ffb703]")

Initial prompt: What is the weather in New York?

Intermediate response: Okay, let's find the weather in New York:

Tool name: get_lat_long

Tool input: {'place': 'New York'}

Tool output: {'latitude': '40.7127281', 'longitude': '-74.0060152'}

Intermediate response: <thinking>
Now that we have the latitude and longitude for New York, we can use that to look up the current weather.
</thinking>

Tool name: get_weather

Tool input: {'latitude': '40.7127281', 'longitude': '-74.0060152'}

Tool output: {'latitude': 40.710335, 'longitude': -73.99309, 'generationtime_ms': 0.04398822784423828, 
'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 32.0, 
'current_weather_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature': '°C', 'windspeed': 'km/h', 
'winddirection': '°', 'is_day': '', 'weathercode': 'wmo code'}, 'current_weather': {'time': '2024-08-25T17:45', 
'interval': 900, 'temperature': 29.4, 'windspeed': 12.5, 'winddirection': 192, 'is_day': 1, 'weathercode': 0}}

Final answer: According to the weather data, the current weather in New York is:
- Temperature: 29.4°C
- Wind speed: 12.5 km/h
- Wind direction: 192°
- Weather code: 0 (clear sky)

---
## Next steps

Now that you have used a few different retrieval systems, lets move on to the next notebook where you can apply the skills you've learned so far!